In [ ]:
for i in range(10) 
    A= liste vide année 
    V= liste vide 
    calcul pr l'année i 
    A.append (2023-i ; ......) -> l'inverser pour commencer par 2002 
    V.append( volatilité)

et ensuite plot (matplotlib ou pilot) de A et V 

In [92]:
import pandas as pd
import numpy as np
import yfinance as yf

A=[]
V=[]
for i in range(10):
    data = yf.download("^FCHI", start=str(2023-i-1)+"-11-30", end=str(2023-i)+"-11-30", interval = "1d")
    returns = data['Close'].pct_change(fill_method=None).dropna()
    hist_vol = np.sqrt(len(returns)) * returns.std()
    A.append(2023-i)
    V.append(hist_vol)
A=A[::-1]
V=V[::-1]
print(A)
print(V)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
[0.15242382441041916, 0.22649493117050784, 0.21881236309248925, 0.10597372177224804, 0.1278805153596491, 0.1426213943689208, 0.32518246094511893, 0.13615442431967675, 0.22603697667382122, 0.14633236455915308]
